In [2]:
!pip install streamlit transformers --quiet



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.0 MB/s eta 0:00:00


In [5]:

!ngrok config add-authtoken 2zK2X513eJPKy4rQhHdYUx21aIY_sHdVn4GhRSK6Ziyv57Kh


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
code = '''
import streamlit as st
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

# Configuration
st.set_page_config(page_title="LLM Chatbot", page_icon="🤖")
st.title("🤖 LLM Chatbot - Streamlit")

# Charger le modèle LLM
@st.cache_resource
def load_model():
    model_name = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    return pipeline("text2text-generation", model=model, tokenizer=tokenizer)

chatbot = load_model()

# Initialiser l'historique
if "messages" not in st.session_state:
    st.session_state.messages = []

# Afficher l'historique
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# Entrée utilisateur
if prompt := st.chat_input("Pose une question au chatbot..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        response = chatbot(f"Answer as a helpful chatbot: {prompt}", max_new_tokens=100)[0]['generated_text']
        st.markdown(response)

    st.session_state.messages.append({"role": "assistant", "content": response})
'''

with open("chatbot_app.py", "w") as f:
    f.write(code)


In [6]:
from pyngrok import ngrok

# Ouvrir un tunnel sur le port 8501
public_url = ngrok.connect(8501)
print(f"🌐 Votre app Streamlit est disponible ici : {public_url}")



🌐 Votre app Streamlit est disponible ici : NgrokTunnel: "https://ce90-34-80-206-74.ngrok-free.app" -> "http://localhost:8501"


In [8]:
!streamlit run chatbot_app.py &> /dev/null &
